In [ ]:
# Libraries 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from PIL import Image
import os


# changer les noms des fichiers "image" pour pouvoir les lire


In [ ]:
import os
classes = 43
for i in range(classes) :
    path = "data/train/{0}/".format(i)
    print(path)
    Class=os.listdir(path)
    j=0
    for a in Class:
        if(a[0] != "0"):            
            filepath = path+str(i)+str(j)+"."+a[-3:]
            os.rename(path+'\\'+a,path+str(i)+str(j)+"."+a[-3:])
            j=j+1


# Lire les données

In [ ]:
# Reading the input images and putting them into a numpy array
data=[]
labels=[]

height = 30
width = 30
channels = 3
classes = 43
n_inputs = height * width*channels

for i in range(classes) :
    path = "data/train/{0}/".format(i)
    print(path)
    Class=os.listdir(path)
    for a in Class:
        try:
            image=cv2.imread(path+'\\'+a)
            # print(a)
            image_from_array = Image.fromarray(image, 'RGB')
            size_image = image_from_array.resize((height, width))
            data.append(np.array(size_image))
            labels.append(i)
        except AttributeError:
            print(" ")
            
Cells=np.array(data)
labels=np.array(labels)

#Randomize the order of the input images
s=np.arange(Cells.shape[0])
np.random.seed(43)
np.random.shuffle(s)
Cells=Cells[s]
labels=labels[s]

In [ ]:
#Spliting the images into train and validation sets
X_train=Cells
X_train = X_train.astype('float32')/255 
y_train=labels

#Using one hote encoding for the train and validation labels
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, 43)

In [ ]:
y_train[23][5]

In [ ]:
#Definition of the DNN model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

#Compilation of the model
model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

In [ ]:
#using ten epochs for the training and saving the accuracy for each epoch
epochs = 20
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs)

In [ ]:
history.history.keys()

In [ ]:

#afficher la precision du model et la convergence du model
import matplotlib.pyplot as plt

plt.figure(0)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()

plt.figure(1)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()

In [ ]:

y_test=pd.read_csv("data/Test.csv")
y_test['Path'].to_numpy()

In [ ]:
#tester le model
y_test=pd.read_csv("data/Test.csv")
labels=y_test['Path'].to_numpy()
y_test=y_test['ClassId'].values

data=[]

for f in labels:
    image=cv2.imread('data/test/'+f.replace('Test/', ''))
    image_from_array = Image.fromarray(image, 'RGB')
    size_image = image_from_array.resize((height, width))
    data.append(np.array(size_image))

X_test=np.array(data)
X_test = X_test.astype('float32')/255 
pred = model.predict(X_test)

In [ ]:
# afficher la precision du model en utilisant le test
from sklearn.metrics import accuracy_score
p = pred.copy()
p.shape
p
jdida = []
for i in p:
    jdida.append(np.argmax(np.array(i)))

jdida
accuracy_score(y_test, jdida)


In [ ]:
# sauvgarder le model pour des utilisations ulterieures
model.save('tp_imn_detection_panneau.h5')

In [2]:
import tkinter as tk
from tkinter import filedialog
from tkinter import *
from PIL import ImageTk, Image
import cv2
import numpy as np
height = 30
width = 30
#charger le model entrainé pour la detection de panneau
from tensorflow.keras.models import load_model
model = load_model('tp_imn_detection_panneau.h5')
#dictionnaire pour étiqueter toutes les classes de panneaux de signalisation.
classes = { 1:'Limite de vitesse (20km/h)',
           2:'Limite de vitesse (30km/h)',
           3:'Limitation de vitesse (50km/h)',
           4:'Limite de vitesse (60km/h)',
           5:'Limite de vitesse (70km/h)',
           6:'Limite de vitesse (80km/h)',
           7:'Fin limitation de vitesse (80km/h)',
           8:'Limite de vitesse (100km/h)',
           9:'Limite de vitesse (120km/h)',
           10:'Pas de dépassement',
           11:'Interdiction de dépasser les véhicules de plus de 3,5 tonnes',
           12:'Droit de passage à l\'intersection',
           13:'Route prioritaire',
           14:'Cédez le passage',
           15:'Stop',
           16:'Pas de véhicules',
           17:'Véh > 3,5 tonnes interdits',
           18:'Pas d\'entrée',
           19:'Précaution générale',
           20:'Courbe dangereuse à gauche',
           21:'Courbe dangereuse à droite',
           22:'Double courbe',
           23:'Route cahoteuse',
           24:'Route glissante',
           25:'La route se rétrécit à droite',
           26:'Travaux routiers',
           27:'Signaux de circulation',
           28:'Piétons',
           29:'Enfants traversant',
           30:'Passage à vélo',
           31:'Attention à la glace/neige',
           32:'Animaux sauvages traversant',
           33:'Vitesse finale + limites de dépassement',
           34:'Tourne à droite',
           35:'Tourner à gauche devant',
           36: 'En avant seulement',
           37:'Aller tout droit ou à droite',
           38:'Aller tout droit ou à gauche',
           39:'Tenir à droite',
           40:'Tenir à gauche',
           41:' Rond-point obligatoire',
           42:'Fin du non-passage',
           43:'Aucun passage de véhicule d\'un poids supérieur à 3,5tonnes' }

# interface graphique
top=tk.Tk()
top.geometry('800x600')
top.title('Traffic sign classification')
top.configure(background='#CDCDCD')
label=Label(top,background='#CDCDCD', font=('arial',15,'bold'))
sign_image = Label(top)
def classify(file_path):
   global label_packed
   image=cv2.imread(file_path)
   print(type(image))
   image_from_array = Image.fromarray(image, 'RGB')
   size_image = image_from_array.resize((height, width))
   exe = []
   exe.append(np.array(size_image))
   X_test1=np.array(exe)
   X_test1 = X_test1.astype('float32')/255 
   pred = model.predict(X_test1)
   print(X_test1.shape[1:])
   pred = np.argmax(pred,axis=1)   
   
   #####################################   
   
   sign = classes[pred[0]+1]
   print(sign)
   label.configure(foreground='red', text=sign)
def show_classify_button(file_path):
   classify_b=Button(top,text="Classer le panneau",command=lambda: classify(file_path),padx=10,pady=5)
   classify_b.configure(background='#364156', foreground='white',font=('arial',10,'bold'))
   classify_b.place(relx=0.79,rely=0.46)
def upload_image():
    try:
        file_path=filedialog.askopenfilename()
        uploaded=Image.open(file_path)
        uploaded.thumbnail(((top.winfo_width()/2.25),(top.winfo_height()/2.25)))
        im=ImageTk.PhotoImage(uploaded)
        sign_image.configure(image=im)
        sign_image.image=im
        label.configure(text='')
        show_classify_button(file_path)
    except:
        pass
upload=Button(top,text="Télécharger une image",command=upload_image,padx=10,pady=5)
upload.configure(background='#364156', foreground='white',font=('arial',10,'bold'))
upload.pack(side=BOTTOM,pady=50)
sign_image.pack(side=BOTTOM,expand=True)
label.pack(side=BOTTOM,expand=True)
heading = Label(top, text="Vérifier le panneau de signalisation",pady=20, font=('arial',20,'bold'))
heading.configure(background='#CDCDCD',foreground='#364156')
heading.pack()
top.mainloop()

<class 'numpy.ndarray'>
(30, 30, 3)
Tenir à droite
